In [44]:
import tensorflow as tf
import numpy as np
import pandas as pd

In [45]:
ds = pd.read_csv('topSongsLyrics1950_2019.csv')

In [46]:
ds.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 700 entries, 0 to 699
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   year    700 non-null    int64 
 1   rank    700 non-null    int64 
 2   artist  700 non-null    object
 3   song    700 non-null    object
 4   lyrics  696 non-null    object
dtypes: int64(2), object(3)
memory usage: 27.5+ KB


In [47]:
X = ds['lyrics']

In [48]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, GRU, Dense, Dropout
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import nltk

In [49]:
voc_size = 8000

In [50]:
messages = X.copy()

In [51]:
import nltk

nltk.download("punkt")
nltk.download("punkt_tab")   # <-- new requirement

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\aditya\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\aditya\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


True

In [52]:
from nltk.tokenize import sent_tokenize

# convert to string and drop NaN
text = " ".join(messages.dropna().astype(str))

# tokenize sentences
corpus = sent_tokenize(text)

print(corpus[:5])  # first 5 sentences

["They call, they call me the fat man|'Cause I weight two hundred pounds|All the girls, they love me|'Cause I know my way around|I was standin', I was standin' on the corner|Of Rampart and Canal|I was watchin', watchin'|Watchin' all these creole gals|Wah wah wah, wah wah|Wah wah waah, wah wah wah|Wah wah waah, wah wah wah|Wah wah wah|Wah waaa-ah wah|Wah wah wah, wah wah wah|Wah wah wah, wah wah wah|Wah wah wah|I'm goin', I'm goin' goin' away|And I'm goin', goin' to stay|'Cause women and a bad life|They're carrying this soul awayYou might also like Understanding and peace of mind|But, if it's not asking too much|Please send me someone to love|Show all the world how to get along|Peace will enter when hate is gone|But, if it's not asking too much|Please send me someone to love|I lay awake night and ponder world troubles|My answer is always the same|That unless men put an end to all of this|Hate will put the world in a flame, (oh) what a shame|Just because I'm in misery|I'm not begging for

In [53]:
corpus = []
for lyric in messages.dropna():
    sentences = sent_tokenize(str(lyric))
    corpus.extend(sentences)  # add all sentences to corpus

print(corpus[:5])

["They call, they call me the fat man|'Cause I weight two hundred pounds|All the girls, they love me|'Cause I know my way around|I was standin', I was standin' on the corner|Of Rampart and Canal|I was watchin', watchin'|Watchin' all these creole gals|Wah wah wah, wah wah|Wah wah waah, wah wah wah|Wah wah waah, wah wah wah|Wah wah wah|Wah waaa-ah wah|Wah wah wah, wah wah wah|Wah wah wah, wah wah wah|Wah wah wah|I'm goin', I'm goin' goin' away|And I'm goin', goin' to stay|'Cause women and a bad life|They're carrying this soul awayYou might also like", "Understanding and peace of mind|But, if it's not asking too much|Please send me someone to love|Show all the world how to get along|Peace will enter when hate is gone|But, if it's not asking too much|Please send me someone to love|I lay awake night and ponder world troubles|My answer is always the same|That unless men put an end to all of this|Hate will put the world in a flame, (oh) what a shame|Just because I'm in misery|I'm not begging 

In [54]:
# Keep only 50k–100k lines of text (enough to train)
lyrics_text = " ".join(ds["lyrics"].dropna().tolist()[:50000])

In [55]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(corpus)
total_words = len(tokenizer.word_index) + 1

In [56]:
input_sequences = []
for line in corpus:
    token_list = tokenizer.texts_to_sequences([line])[0]
    for i in range(3, len(token_list)):   # sequence length >= 3
        n_gram_seq = token_list[:i+1]
        input_sequences.append(n_gram_seq)

In [57]:
max_len = max([len(x) for x in input_sequences])
input_sequences = pad_sequences(input_sequences, maxlen=max_len, padding="pre")

X = input_sequences[:, :-1]
y = input_sequences[:, -1].astype("int32")   # keep labels as integers (no one-hot)


In [58]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout

vocab_size = 5000
seq_length = 50
max_sequence_len = max_len - 1
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense

model = Sequential([
    Embedding(vocab_size, 128, input_length=seq_length),
    LSTM(256, return_sequences=True),  # try 128 instead of 512/1024
    LSTM(256),
    Dense(vocab_size, activation="softmax")
])

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.build(input_shape=(None, max_sequence_len))
model.summary()

C:\Users\aditya\AppData\Roaming\Python\Python312\site-packages\keras\src\layers\core\embedding.py:97: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "sequential_4"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_4 (Embedding)         │ (None, 1774, 128)      │       640,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_8 (LSTM)                   │ (None, 1774, 256)      │       394,240 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_9 (LSTM)                   │ (None, 256)            │       525,312 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 5000)           │     1,285,000 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,844,552 (10.85 MB)

 Trainable params: 2,844,552 (10.85 MB)

 Non-trainable params: 0 (0.00 B)

In [59]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout

# 1. Load dataset and take only half
df = pd.read_csv("topSongsLyrics1950_2019.csv")
half_len = len(df) // 2
lyrics = df["lyrics"].dropna().astype(str).tolist()[:half_len]

# 2. Preprocess text
corpus = [lyric.lower() for lyric in lyrics]

# 3. Tokenizer with limited vocab
vocab_size = 5000   # keep vocab smaller
tokenizer = Tokenizer(num_words=vocab_size, oov_token="<OOV>")
tokenizer.fit_on_texts(corpus)

# 4. Create sequences
input_sequences = []
for line in corpus:
    token_list = tokenizer.texts_to_sequences([line])[0]
    for i in range(1, len(token_list)):
        n_gram_seq = token_list[:i+1]
        input_sequences.append(n_gram_seq)

# Pad sequences (cap sequence length at 20 for speed)
max_seq_len = min(20, max(len(x) for x in input_sequences))
input_sequences = pad_sequences(input_sequences, maxlen=max_seq_len, padding='pre')

X, y = input_sequences[:, :-1], input_sequences[:, -1]
y = tf.keras.utils.to_categorical(y, num_classes=vocab_size)

# 5. Build smaller LSTM model
model = Sequential([Embedding(vocab_size, 128), # Removed input_length
  LSTM(256, return_sequences=True),  # try 128 instead of 512/1024
  Dropout(0.2),
  LSTM(256),
  Dropout(0.2),
  Dense(vocab_size, activation="softmax")
])
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# 6. Train model (few epochs to prevent crash)
model.fit(X, y, epochs=10, batch_size=128, verbose=1)

Epoch 1/10
816/816 ━━━━━━━━━━━━━━━━━━━━ 126s 150ms/step - accuracy: 0.0329 - loss: 6.6641
Epoch 2/10
816/816 ━━━━━━━━━━━━━━━━━━━━ 125s 153ms/step - accuracy: 0.0529 - loss: 6.0126
Epoch 3/10
816/816 ━━━━━━━━━━━━━━━━━━━━ 124s 152ms/step - accuracy: 0.0766 - loss: 5.6737
Epoch 4/10
816/816 ━━━━━━━━━━━━━━━━━━━━ 120s 146ms/step - accuracy: 0.0910 - loss: 5.4678
Epoch 5/10
816/816 ━━━━━━━━━━━━━━━━━━━━ 127s 156ms/step - accuracy: 0.1079 - loss: 5.2417
Epoch 6/10
816/816 ━━━━━━━━━━━━━━━━━━━━ 129s 158ms/step - accuracy: 0.1243 - loss: 5.0385
Epoch 7/10
816/816 ━━━━━━━━━━━━━━━━━━━━ 110s 135ms/step - accuracy: 0.1459 - loss: 4.8337
Epoch 8/10
816/816 ━━━━━━━━━━━━━━━━━━━━ 117s 144ms/step - accuracy: 0.1628 - loss: 4.6550
Epoch 9/10
816/816 ━━━━━━━━━━━━━━━━━━━━ 123s 150ms/step - accuracy: 0.1845 - loss: 4.4681
Epoch 10/10
816/816 ━━━━━━━━━━━━━━━━━━━━ 121s 149ms/step - accuracy: 0.2009 - loss: 4.3055


In [60]:
def predict_next_word(seed_text, next_words=10):
    for _ in range(next_words):
        token_list = tokenizer.texts_to_sequences([seed_text])[0]
        token_list = pad_sequences([token_list], maxlen=max_seq_len-1, padding='pre')
        predicted = np.argmax(model.predict(token_list, verbose=0), axis=-1)[0]
        for word, index in tokenizer.word_index.items():
            if index == predicted:
                seed_text += " " + word
                break
    return seed_text

# Example
print(predict_next_word("love is"))

love is a little bit i want to be my life i
